<a href="https://colab.research.google.com/github/stefvas/neural-networks/blob/main/Unsupervised_Learnring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Εργαστηριακή Άσκηση 2. Μη επιβλεπόμενη μάθηση. 
## Σύστημα συστάσεων βασισμένο στο περιεχόμενο
## Σημασιολογική απεικόνιση δεδομένων με χρήση SOM 

In [ ]:
!pip install --upgrade pip
!pip install --upgrade numpy
!pip install --upgrade pandas
!pip install --upgrade nltk
!pip install --upgrade scikit-learn
!pip install --upgrade joblib

     |████████████████████████████████| 1.7 MB 5.5 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
     |████████████████████████████████| 15.7 MB 5.0 MB/s            
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.3.post1 requires numpy<1.20,>=1.16.0, but you have numpy 1.21.5 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


     |████████████████████████████████| 11.3 MB 5.5 MB/s            
  Attempting uninstall: pandas
    Found existing installation: pandas 1.1.5
    Uninstalling pandas-1.1.5:
      Successfully uninstalled pandas-1.1.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas~=1.1.0; python_version >= "3.0", but you have pandas 1.3.5 which is incompatible.


     |████████████████████████████████| 1.5 MB 3.8 MB/s            
     |████████████████████████████████| 749 kB 22.4 MB/s            
  Attempting uninstall: regex
    Found existing installation: regex 2019.12.20
    Uninstalling regex-2019.12.20:
      Successfully uninstalled regex-2019.12.20
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


##Data insertion

In [ ]:
import pandas as pd

dataset_url = "https://drive.google.com/uc?export=download&id=1-tN5rC1QCgwqL67cvIXdMbtd-Xf9Xqow"
df_data_1 = pd.read_csv(dataset_url, sep='\t',  header=None, quoting=3)

Team seed == 40

In [ ]:
import numpy as np

# Στο επόμενη γραμή βάλτε τον αριθμό της ομάδας στο εργαστήριο των νευρωνικών
team_seed_number = 40

movie_seeds_url = "https://drive.google.com/uc?export=download&id=1-oeMf7BFmiH70XObgC9WOTo-a9cDJZwP"
df_data_2 = pd.read_csv(movie_seeds_url, header=None)

# επιλέγεται 
my_index = df_data_2.iloc[team_seed_number,:].values

titles = df_data_1.iloc[:, [2]].values[my_index] # movie titles (string)
categories = df_data_1.iloc[:, [3]].values[my_index] # movie categories (string)
bins = df_data_1.iloc[:, [4]]
catbins = bins[4].str.split(',', expand=True).values.astype(float)[my_index] # movie categories in binary form (1 feature per category)
summaries =  df_data_1.iloc[:, [5]].values[my_index] # movie summaries (string)
corpus = summaries[:,0].tolist() # list form of summaries
corpus_df = pd.DataFrame(corpus) # dataframe version of corpus


In [ ]:
#print(type(corpus))
punc = '''!@#$%^&*(){}[];:?/.,\|`~_-'''
def simplify_lower(arr):
  corpus_prime = []
  for string in arr:
    temp  = ""
    for char in string:
      if (char.isupper()):
         char = char.lower()
      temp += char
    string = temp
    corpus_prime.append(string)
  return corpus_prime


def simplify_punc(arr):
  corpus_prime = []
  for string in arr:
    temp  = ""
    for char in string:
      if(char in punc):
       count = 0
      else:
        temp += char
    string = temp
    corpus_prime.append(string)
  return corpus_prime

corpus = simplify_lower(corpus)

corpus = simplify_punc(corpus)





In [ ]:
ID = 999
print(titles[ID])
print(categories[ID])
print(catbins[ID])
print(corpus[ID])

convict 99
"comedy",  "black-and-white"
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0

# Εφαρμογή 1. Υλοποίηση συστήματος συστάσεων ταινιών βασισμένο στο περιεχόμενο
<img src="http://clture.org/wp-content/uploads/2015/12/Netflix-Streaming-End-of-Year-Posts.jpg" width="70%">

##TFID convertion

In [ ]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS as stopwords_english
from sklearn.feature_extraction.text import TfidfVectorizer


def tf_idf(mx_df, mn_df, my_stopwrods=list()):

  stopwords_film = ["movie", "film", "plot", "begins", "opens", "starts", "piece", "named", "woman", "women", "man", "men", "prologue", "help", "helping"]
  stopwords = list(stopwords_english) + list(my_stopwrods) + list(stopwords_film)# enhancing stopwords

  vectorizer = TfidfVectorizer(max_df=mx_df, min_df=mn_df, stop_words=stopwords, analyzer='word')
  vectorizer.fit(corpus)
  corpus_tf_idf = vectorizer.transform(corpus)
  print("Dimensions: ", corpus_tf_idf.shape, "\n")


  return corpus_tf_idf

Η συνάρτηση [TfidfVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) όπως καλείται εδώ **δεν είναι βελτιστοποιημένη**. Οι επιλογές των μεθόδων και παραμέτρων της μπορεί να έχουν **δραματική επίδραση στην ποιότητα των συστάσεων** και είναι διαφορετικές για κάθε dataset. Επίσης, οι επιλογές αυτές έχουν πολύ μεγάλη επίδραση και στη **διαστατικότητα και όγκο των δεδομένων**. Η διαστατικότητα των δεδομένων με τη σειρά της θα έχει πολύ μεγάλη επίδραση στους **χρόνους εκπαίδευσης**, ιδιαίτερα στη δεύτερη εφαρμογή της άσκησης. Ανατρέξτε στα notebooks του εργαστηρίου και στο [FAQ](https://docs.google.com/document/d/1-E4eQkVnTxa3Jb0HL9OAs11bugYRRZ7RNWpu7yh9G4s/edit?usp=sharing) των ασκήσεων.

In [ ]:
print(tf_idf(1.0, 0))

Dimensions:  (5000, 52539) 

  (0, 52165)	0.018276263143360158
  (0, 52164)	0.011302700569169849
  (0, 52015)	0.03264021889323426
  (0, 51882)	0.03172604941068616
  (0, 51792)	0.033145149318060466
  (0, 51780)	0.022940377239836447
  (0, 51768)	0.024890518157703857
  (0, 51705)	0.023741574906689406
  (0, 51698)	0.013514636775153157
  (0, 51499)	0.05481237438741874
  (0, 51420)	0.037515940318718785
  (0, 51392)	0.022261513445763933
  (0, 51388)	0.2966426822230622
  (0, 51313)	0.023054645553730675
  (0, 50841)	0.019551640845749688
  (0, 50726)	0.01358319535923186
  (0, 50722)	0.03993165835513382
  (0, 50530)	0.02272027225231764
  (0, 50380)	0.04230475576878638
  (0, 49619)	0.0300563329418967
  (0, 49556)	0.03094744890864895
  (0, 49548)	0.01589422568557242
  (0, 49540)	0.015701187246722453
  (0, 49087)	0.030651378213432518
  (0, 49020)	0.023809610122387907
  :	:
  (4999, 9443)	0.01938413548731317
  (4999, 8978)	0.039876128544933144
  (4999, 8554)	0.02015269020706517
  (4999, 8548)	0.01672

#Suggestion system implementation

Το σύστημα συστάσεων που θα υλοποιήσετε θα είναι μια συνάρτηση content_recommender με τρία ορίσματα: target_movie, max_recommendations και corpus_type. Στην target_movie περνάμε το ID μιας ταινίας-στόχου για την οποία μας ενδιαφέρει να βρούμε παρόμοιες ως προς το περιεχόμενο (τη σύνοψη) ταινίες, max_recommendations στο πλήθος.
Υλοποιήστε τη συνάρτηση ως εξής:

    Για την ταινία-στόχο, θα υπολογίζετε την ομοιότητα συνημιτόνου της με όλες τις ταινίες της συλλογής σας όπως αυτές αναπαριστώνται στο corpus_type.με βάση την ομοιότητα συνημιτόνου που υπολογίσατε, δημιουργήστε ταξινομημένο πίνακα από το μεγαλύτερο στο μικρότερο, με τα indices (ID) των ταινιών.
    
    Για την ταινία-στόχο εκτυπώστε: id, τίτλο, σύνοψη, κατηγορίες (categories)
    
    Για τις max_recommendations ταινίες (πλην της ίδιας της ταινίας-στόχου που έχει cosine similarity 1 με τον εαυτό της) με τη μεγαλύτερη ομοιότητα συνημιτόνου (σε φθίνουσα σειρά), τυπώστε σειρά σύστασης (1 πιο κοντινή, 2 η δεύτερη πιο κοντινή κλπ), ομοιότητα συνημιτόνου, id, τίτλο, σύνοψη, και κατηγορίες (categories)


In [ ]:
# function that prints movie info
def print_movie(ID):
  print("ID: ", ID)
  print("Title: ", ', '.join(titles[ID]))
  print("Categories: ", ', '.join(categories[ID]))
  print("Summary:\n", corpus[ID], "\n")

In [ ]:
from sklearn.metrics.pairwise import linear_kernel

def content_recommender(ID, max_recommendations, corpus_tf_idf):
  cosine_similarities = linear_kernel(corpus_tf_idf[ID], corpus_tf_idf).flatten()  # calculating cosine similarities using sparse matrices

  cosine_similarities = (-cosine_similarities).argsort()  # sorting the array and saving movies based on ID's

  print_movie(ID)  # print target movie info

  for i in range(1, max_recommendations+1):   # print recommendations
    print("Place:", i)
    print_movie(cosine_similarities[i])

In [ ]:
corpus_type = tf_idf(1.0, 0)

Dimensions:  (5000, 52539) 



In [ ]:
content_recommender(1, 5, corpus_type)

ID:  1
Title:  r, e, s, t, l, e, s, s
Categories:  ", c, o, m, e, d, y, ", ,,  ,  , ", w, o, r, l, d,  , c, i, n, e, m, a, ", ,,  ,  , ", c, h, i, n, e, s, e,  , m, o, v, i, e, s, "
Summary:
 leah  is a translator who is traveling international cities in search of romance and closure with her estranged boyfriend jeff she eventually settles in beijing china where she meets and develops a fixation on master sun zhan  who teaches here a chinese game called weiqi when her relationship with master sun zhan takes a turn for the worse leah decides to focus on getting revenge on jeff richard  is a shiftless chineseamerican whose family sends him to beijing with his grandfather's ashes eventually leah and richard's paths cross and a romance develops 

Place: 1
ID:  2737
Title:  e, y, e, s,  , o, f,  , f, i, r, e
Categories:  ", h, o, r, r, o, r, "
Summary:
 the film takes place in the year 1750 on the american frontier during the colonial days before the united states declared its independence 

#Utilization

Αφού υλοποιήσετε τη συνάρτηση content_recommender χρησιμοποιήστε την για να βελτιστοποιήσετε την TfidfVectorizer. Συγκεκριμένα, αρχικά μπορείτε να δείτε τι επιστρέφει το σύστημα για τυχαίες ταινίες-στόχους και για ένα μικρό max_recommendations (2 ή 3). Αν σε κάποιες ταινίες το σύστημα μοιάζει να επιστρέφει σημασιολογικά κοντινές ταινίες σημειώστε το ID τους. Δοκιμάστε στη συνέχεια να βελτιστοποιήσετε την TfidfVectorizer για τα συγκεκριμένα ID ώστε να επιστρέφονται σημασιολογικά κοντινές ταινίες για μεγαλύτερο αριθμό max_recommendations. Παράλληλα, όσο βελτιστοποιείτε την TfidfVectorizer, θα πρέπει να λαμβάνετε καλές συστάσεις για μεγαλύτερο αριθμό τυχαίων ταινιών.

Ταυτόχρονα, μια αντίρροπη κατά κάποιο τρόπο κατεύθυνση της βελτιστοποίησης είναι να χρησιμοποιείτε τις παραμέτρους του TfidfVectorizer έτσι ώστε να μειώνονται οι διαστάσεις του Vector Space Model μέχρι το σημείο που θα αρχίσει να εμφανίζονται επιπτώσεις στην ποιότητα των συστάσεων.

In [ ]:
def similar_movies(count):
  movie_list = []

  for i in range(0, 5000): # for all movies
    cnt = count

    cosine_similarities = linear_kernel(corpus_tf_idf[i], corpus_tf_idf).flatten()  # find best matches
    cosine_similarities = (-cosine_similarities).argsort()
    
    categs = categories[i][0]
    categs = categs.split(",")
    
    for category in categs: # find categories similar between th movie and the 3 top matches
      if category.lstrip(' ') in (categories[cosine_similarities[1]][0]) and category.lstrip(' ') in (categories[cosine_similarities[2]][0]) and category.lstrip(' ') in (categories[cosine_similarities[3]][0]):
        cnt = cnt - 1
      if cnt == 0: # if count similar categories are found append the movie to movie_list
        movie_list.append(i)
        break

  return movie_list

we add to stop words list English names to avoid the resemblansh of the movies been based on common names

In [ ]:
import urllib.request 
import string

# female english names
target_url = "http://www.cs.cmu.edu/Groups/AI/util/areas/nlp/corpora/names/female.txt"
response = urllib.request.urlopen(target_url).read().decode()
lines_temp = response.split('\n')
names = []

for line in lines_temp:
  if line and line[0] != "#":
    names.append(line.lower())

# male english names
target_url = "http://www.cs.cmu.edu/Groups/AI/util/areas/nlp/corpora/names/male.txt"
response = urllib.request.urlopen(target_url).read().decode()
lines_temp = response.split('\n')

for line in lines_temp:
  if line and line[0] != "#":
    names.append(line.lower())


In [ ]:
corpus_tf_idf = tf_idf(1.0, 0, names)

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['luc', 'mar', 'zsa'] not in stop_words.
  % sorted(inconsistent)


Dimensions:  (5000, 49545) 



We see a decrease at the collumns , about 3000

In [ ]:
content_recommender(10, 3, corpus_tf_idf)

ID:  10
Title:  c, o, m, o,  , e, s, q, u, e, c, e, r
Categories:  ", d, r, a, m, a, "
Summary:
 julia  is a professor of english literature who is abandoned by her girlfriend after a relationship that lasted over ten years because of the separation julie moves to rio de janeiro and lives with her best friend hugo  who is gay and lisa  in the new work julia ends up attracting the interest of two students and it is helena  who is after the teacher although she did not feel prepared for a new relationship 

Place: 1
ID:  4711
Title:  o, n, l, y,  , w, h, e, n,  , i,  , d, a, n, c, e
Categories:  ", d, o, c, u, m, e, n, t, a, r, y, "
Summary:
 only when i dance tells the story of two teenagers irlan and isabela who pursue their dreams of becoming professional ballet dancers as a way to escape the violent slums of rio de janeiro 

Place: 2
ID:  3855
Title:  l, o, v, e,  , i, n,  , b, e, t, w, e, e, n
Categories:  ", r, o, m, a, n, c, e,  , f, i, l, m, ", ,,  ,  , ", d, r, a, m, a, "
Summar

#min_df

Ο αποκλεισμός των λιγότερο συχνών λέξεων βοηθά σε κάποιες περιπτώσεις καθώς αποκλείει σπάνιες λέξεις που δεν προσφέρουν στο νόημα, ενώ σε άλλες κάνει το ίδιο αλλά με λέξεις που συσχετίζουν μικρό αριθμό ταινιών της ίδιας κατηγορίας οδηγώντας σε χειρότερα αποτελέσματα.

In [ ]:
corpus_tf_idf = tf_idf(1.0, 2, names)

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['luc', 'mar', 'zsa'] not in stop_words.
  % sorted(inconsistent)


Dimensions:  (5000, 23131) 



In this case we succeded better results with that method 

In [ ]:
content_recommender(46, 3, corpus_tf_idf)

ID:  46
Title:  r, a, j, a, k, u, m, a, r, a, n
Categories:  ", d, r, a, m, a, "
Summary:
 rajakumaran prabhu  is the son of the village chief vijayakumar  yuvaraj  hates rajakumaran and his village selvi  and vaidehi  are in love with their cousin rajakumaran but he chooses vaidehi selvi decides to sacrifice her love her father selvaraj  refuses to give his property to village people as promised and his only request is that rajakumaran gets married with selvi selvi convinced rajakumaran to lie he lies to her father and he gives his property vaidehi committed suicide and rajakumaran reveals his lie one day some goons decide to put a bomb and rajakumaran saves the village the village thinks that the culprit was thangaraj  because he wanted to revenge rajakumaran after his lie angry thangaraj insults rajakumaran's father in public and rajakumaran's father died yuvaraj asks to selvaraj and thangaraj to marry selvi and they accept the village was against the marriage a man  came and said t

In [ ]:
corpus_tf_idf = tf_idf(1.0, 15, names)

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['luc', 'mar', 'zsa'] not in stop_words.
  % sorted(inconsistent)


Dimensions:  (5000, 5525) 



In [ ]:
content_recommender(46, 3, corpus_tf_idf)

ID:  46
Title:  r, a, j, a, k, u, m, a, r, a, n
Categories:  ", d, r, a, m, a, "
Summary:
 rajakumaran prabhu  is the son of the village chief vijayakumar  yuvaraj  hates rajakumaran and his village selvi  and vaidehi  are in love with their cousin rajakumaran but he chooses vaidehi selvi decides to sacrifice her love her father selvaraj  refuses to give his property to village people as promised and his only request is that rajakumaran gets married with selvi selvi convinced rajakumaran to lie he lies to her father and he gives his property vaidehi committed suicide and rajakumaran reveals his lie one day some goons decide to put a bomb and rajakumaran saves the village the village thinks that the culprit was thangaraj  because he wanted to revenge rajakumaran after his lie angry thangaraj insults rajakumaran's father in public and rajakumaran's father died yuvaraj asks to selvaraj and thangaraj to marry selvi and they accept the village was against the marriage a man  came and said t

In this case althouth we see that we don't only get drama recommendations

with continuous trys we conclude that the best result is taken when min-df is 5/6

In [ ]:
corpus_tf_idf = tf_idf(1.0, 6, names)

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['luc', 'mar', 'zsa'] not in stop_words.
  % sorted(inconsistent)


Dimensions:  (5000, 10640) 



In [ ]:
content_recommender(46, 3, corpus_tf_idf)

ID:  46
Title:  r, a, j, a, k, u, m, a, r, a, n
Categories:  ", d, r, a, m, a, "
Summary:
 rajakumaran prabhu  is the son of the village chief vijayakumar  yuvaraj  hates rajakumaran and his village selvi  and vaidehi  are in love with their cousin rajakumaran but he chooses vaidehi selvi decides to sacrifice her love her father selvaraj  refuses to give his property to village people as promised and his only request is that rajakumaran gets married with selvi selvi convinced rajakumaran to lie he lies to her father and he gives his property vaidehi committed suicide and rajakumaran reveals his lie one day some goons decide to put a bomb and rajakumaran saves the village the village thinks that the culprit was thangaraj  because he wanted to revenge rajakumaran after his lie angry thangaraj insults rajakumaran's father in public and rajakumaran's father died yuvaraj asks to selvaraj and thangaraj to marry selvi and they accept the village was against the marriage a man  came and said t

Για να επιβεβαιώσουμε την σωστή τιμή του min_df, η οποία επηρεάζει τη διαστατικότητα του πίνακα και άρα και το χρόνο εκπαίδευσης εκτυπώνουμε τα μεγέθη των λιστών που δίνει η συνάρτηση similar_movies που φτιάξαμε, συναρτήσει της min_df, για αριθμό κοινών ειδών μεταξύ των ταινιών 2 και 3.

In [ ]:
import matplotlib.pyplot as plt

length_2 = []
length_3 = []
min_dfs = [1,2,3,4,5,6,7,8,9,10]
for min_df in min_dfs:
  corpus_tf_idf = tf_idf(1.0, min_df, names)
  temp = []
  length_2.append(len(similar_movies(2)))
  length_3.append(len(similar_movies(3)))
  

plt.plot(min_dfs, length_2)
plt.plot(min_dfs, length_3)
plt.xlabel('min_df')
plt.ylabel('Number of Similar Movies')
plt.title('Recommendations Quality to min_df')
plt.legend(['2 same genres', '3 same genres'], loc='center right')
plt.show()

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['luc', 'mar', 'zsa'] not in stop_words.
  % sorted(inconsistent)


Dimensions:  (5000, 49545) 



/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['luc', 'mar', 'zsa'] not in stop_words.
  % sorted(inconsistent)


Dimensions:  (5000, 23131) 



In [ ]:
print(length_2)
print(length_3)

Παρατηρούμε γενικά μείωση της ποιότητας των επιλογών με την αύξηση του min_df, με μία μικρή αύξηση όμως στις τιμές για count = 2 και min_df μεταξύ 4 και 8. Για count = 3 παρατηρείται μείωση της ποιότητας. Έτσι οι παρατηρήσεις μας επιβεβαιώνονται.

#10 Good recommendations

In [ ]:
corpus_tf_idf = tf_idf(1.0, 6, names)

In [ ]:
content_recommender(12, 5, corpus_tf_idf)

In [ ]:
content_recommender(856, 5, corpus_tf_idf)

In [ ]:
content_recommender(4814, 5, corpus_tf_idf)

In [ ]:
content_recommender(4912, 6, corpus_tf_idf)

In [ ]:
content_recommender(60, 5, corpus_tf_idf)

In [ ]:
content_recommender(455, 4, corpus_tf_idf)

In [ ]:
content_recommender(248, 6, corpus_tf_idf)

In [ ]:
content_recommender(749, 5, corpus_tf_idf)

In [ ]:
content_recommender(1446, 6, corpus_tf_idf)

## Tip: persistence αντικειμένων με joblib.dump


In [ ]:
import joblib

joblib.dump(corpus_tf_idf, 'corpus_tf_idf.pkl', compress=True) 

In [ ]:
!ls -lh

In [ ]:
corpus_tf_idf = joblib.load('corpus_tf_idf.pkl')